## Import Libraries


In [122]:
import nltk
import os
import pandas as pd
import re
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.snowball import SnowballStemmer

## Confirm Directory

In [123]:
os.getcwd()
#os.chdir('/Users/gml/Desktop')
#path1 ='/Users/gml/Desktop'

'/Users/gml/Desktop'

## Read Data from CSV into a Data Frame

In [174]:
data = pd.read_csv('MovieDataSAMPLE_labeledVERYSMALL.csv', delimiter='\n', header = None)
data = data.drop(0, axis = 0)
data = data.rename(columns={0: 'Comments'})
data["Label"] = np.nan
#print(data.head())
print(data.iloc[2,0])

'i\'m really starting to wonder about alicia silverstone . \nsure , she is one of the most beautiful creatures on god\'s green earth ( second only to that movie critic at large guy ) , but when it comes to choosing what movies she stars in , she always strikes out . \nthe crush was a slow-moving , predictable piece of fluff . \nhideaway was a horrific novel adaptation alicia had only a minor role in . \nclueless was an annoying , unfunny waste of time . \nand people have e-mailed me too ," saying clueless is a good movie and that i\'m the only one who doesn\'t like it . \none girl said if i\'d seen the movie with \"" an open mind "," \"" i would have enjoyed it . \nnothing could be further from the truth . \ni went into the theater expecting to love the movie . \nthe preview looked good and of course i\'m crazymadinlove with alicia ", but the movie was a bunch of bad jokes coming from whiny , unlikable characters . \nalmost everyone i saw the movie with felt the same way . \nwhen we we

In [166]:
for row in data.index:
    a = data.iloc[row-1,0]
    if a[-1] == 's':
        data.iloc[row-1,0] = a[0:len(a)-4]
        data.iloc[row-1,1] = 'Positive'
    else:
        data.iloc[row-1,0] = a[0:len(a)-4]
        data.iloc[row-1,1] = 'Negative'

RowNames = "Review #" + data.index.map(str) + ': ' + data["Label"]
print(data.head())

                                            Comments     Label
1  'let\'s say you live at the end of an airport ...  Positive
2  'in this good natured , pleasent and easy goin...  Positive
3  'i\'m really starting to wonder about alicia s...  Negative
4  'so what do you get when you mix together plot...  Negative
5  'the law of crowd pleasing romantic movies sta...  Negative


## Cleaning for the Term Document Matrix

In [167]:
## Uses regual Expressions to find extra texts to remove for later

mylist = data.to_string()
ntl = re.findall(r"\s.\s|[0-9]",mylist)
new_ntl = list(dict.fromkeys(ntl))
issues1 = []
fixes1 = []

print(new_ntl)
for row in new_ntl:
    if not any(a.isalnum() for a in row):
        if row == ' ( ':
            issues1.append(' ( ')
            fixes1.append(' (')
        else:
            issues1.append(row)
            fixes1.append(row[len(row)-2:len(row)])

issues1.append('\\n')
issues1.append('\\t')
issues1.append('\\')

issues2.append('[0-9]')
issues2.append('\s\'$')
issues2.append('^\'')
issues2.append('\,[a-z]*,*$')
issues2.append('\'\"?$')

fixes1.append('')
fixes1.append('')
fixes1.append('')

fixes2.append('')
fixes2.append('')
fixes2.append('')
fixes2.append('')
fixes2.append('')

['   ', '\n1 ', ' . ', ' , ', ' ? ', ' ( ', ' ) ', ' a ', ' i ', ' : ', ' - ', '\n2 ', '1', '9', '8', '4', '\n3 ', '0', '2', '\n4 ', '3', ' ! ', '\n5 ', ' v ']


In [175]:
cleandata1 = pd.Series(data.iloc[:,0])
count = 0
while (count < len(issues1)):
    #print(issues[count])
    cleandata1 = cleandata1.str.replace(issues1[count], fixes1[count], regex=False)
    count = count + 1
count = 0
while (count < len(issues2)):
    #print(issues[count])
    cleandata1 = cleandata1.str.replace(issues2[count], fixes2[count], regex=True)
    count = count + 1
print(cleandata1.iloc[2])

i'm really starting to wonder about alicia silverstone. sure, she is one of the most beautiful creatures on god's green earth (second only to that movie critic at large guy ), but when it comes to choosing what movies she stars in, she always strikes out. the crush was a slow-moving, predictable piece of fluff. hideaway was a horrific novel adaptation alicia had only a minor role in. clueless was an annoying, unfunny waste of time. and people have e-mailed me too ," saying clueless is a good movie and that i'm the only one who doesn't like it. one girl said if i'd seen the movie with "" an open mind "," "" i would have enjoyed it. nothing could be further from the truth. i went into the theater expecting to love the movie. the preview looked good and of course i'm crazymadinlove with alicia ", but the movie was a bunch of bad jokes coming from whiny, unlikable characters. almost everyone i saw the movie with felt the same way. when we were walking out of the theater, one guy (and it wa

## Create Term Document Matrix From a Data Frame

In [151]:
vectorizer = CountVectorizer(ngram_range=(1,3), stop_words='english', max_features = 100)
X = vectorizer.fit_transform(cleandata1)
ColumnNames=vectorizer.get_feature_names()
#print(ColumnNames)
#print(X)

In [152]:
TDM1=pd.DataFrame(X.toarray(),columns=ColumnNames, index = RowNames)
#print(data["Label"])
TDM1.insert(0, "Label", list(data["Label"]), True) 
print(TDM1)

                        Label  actually  alicia  babysitter  bad  bath  best  \
Review #1: Positive  Positive         2       0           0    1     0     1   
Review #2: Positive  Positive         0       0           0    2     0     0   
Review #3: Negative  Negative         0      11           5    3     2     0   
Review #4: Negative  Negative         2       0           0    1     0     1   
Review #5: Negative  Negative         0       0           0    0     1     1   

                     big  bit  buy  ...  time  ve  ve seen  wants  work  \
Review #1: Positive    1    3    3  ...     2   2        2      1     0   
Review #2: Positive    0    2    0  ...     0   0        0      0     0   
Review #3: Negative    0    0    0  ...     1   2        1      0     0   
Review #4: Negative    1    2    0  ...     1   0        0      0     1   
Review #5: Negative    2    0    0  ...     0   2        1      2     2   

                     world  year  zero  zero plus  zero plus zero  


## Read Data from CSV into a Data Frame

In [177]:
data = pd.read_csv('RestaurantSentimentCleanerLABELEDDataSMALLSAMPLE.csv', delimiter='\n', header = None)
data = data.drop(0, axis = 0)
data = data.rename(columns={0: 'Comments'})
data["Label"] = np.nan
#print(data.head(10))
print(data.iloc[0,0])

n,'Mike\'s Pizza High Point, NY Service was very slow and the quality was low. You would think they would know at least how to make good pizza, not. Stick to pre-made dishes like stuffed pasta or a salad. You should consider dining else where.',,


In [178]:
for row in data.index:
    a = data.iloc[row-1,0]
    if a[0] == 'p':
        data.iloc[row-1,0] = a[2:len(a)]
        data.iloc[row-1,1] = 'Positive'
    else:
        data.iloc[row-1,0] = a[2:len(a)]
        data.iloc[row-1,1] = 'Negative'

RowNames = "Review #" + data.index.map(str) + ': ' + data["Label"]
print(data.head(10))

                                            Comments     Label
1  'Mike\'s Pizza High Point, NY Service was very...  Negative
2  'i really like this buffet restaurant in Marsh...  Negative
3  'After I went shopping with some of my friend,...  Negative
4  'Olive Oil Garden was very disappointing. I ex...  Negative
5  'The Seven Heaven restaurant was never known f...  Negative
6  I went to XYZ restaurant and loved it. I had a...  Positive
7  I went to ABC restaurant two days ago and I re...  Positive
8  I went to the Chilis on Erie Blvd and had the ...  Positive
9  OMG. This restaurant is great. The receptionis...  Positive


## Cleaning for the Term Document Matrix

In [179]:
## Uses regual Expressions to find extra texts to remove for later

mylist = data.to_string()
ntl = re.findall(r"\s.\s|[0-9]",mylist)
new_ntl = list(dict.fromkeys(ntl))
issues1 = []
fixes1 = []
issues2 = []
fixes2 = []

#print(new_ntl)
for row in new_ntl:
    if not any(a.isalnum() for a in row):
        if row == ' ( ':
            issues1.append(' ( ')
            fixes1.append(' (')
        else:
            issues1.append(row)
            fixes1.append(row[len(row)-2:len(row)])

issues1.append('\\n')
issues1.append('\\t')
issues1.append('\\')

issues2.append('[0-9]')
issues2.append('\',*$')
issues2.append('^\'')

fixes1.append('')
fixes1.append('')
fixes1.append('')

fixes2.append('')
fixes2.append('')
fixes2.append('')

In [181]:
cleandata2 = pd.Series(data.iloc[:,0])
count = 0
while (count < len(issues1)):
    #print(issues[count])
    cleandata2 = cleandata2.str.replace(issues1[count], fixes1[count], regex=False)
    count = count + 1
count = 0
while (count < len(issues2)):
    #print(issues[count])
    cleandata2 = cleandata2.str.replace(issues2[count], fixes2[count], regex=True)
    count = count + 1
print(cleandata2.iloc[0])

Mike's Pizza High Point, NY Service was very slow and the quality was low. You would think they would know at least how to make good pizza, not. Stick to pre-made dishes like stuffed pasta or a salad. You should consider dining else where.


## Create Term Document Matrix From a Data Frame

In [184]:
vectorizer = CountVectorizer(ngram_range=(1,3), stop_words='english', max_features = 100)
X = vectorizer.fit_transform(cleandata2)
ColumnNames=vectorizer.get_feature_names()
#print(ColumnNames)
#print(X)

In [187]:
TDM2=pd.DataFrame(X.toarray(),columns=ColumnNames, index = RowNames)
#print(data["Label"])
TDM2.insert(0, "Label", list(data["Label"]), True) 
print(TDM2.head(7))

                        Label  abc  appetizer  asked  bring  came  cold  \
Review #1: Negative  Negative    0          0      0      0     0     0   
Review #2: Negative  Negative    0          0      0      0     0     0   
Review #3: Negative  Negative    0          0      0      0     0     0   
Review #4: Negative  Negative    0          0      0      0     0     1   
Review #5: Negative  Negative    0          0      1      2     0     0   
Review #6: Positive  Positive    0          1      1      0     0     0   
Review #7: Positive  Positive    1          0      0      0     1     0   

                     coupon  definitely  delicious  ...  salad  seated  \
Review #1: Negative       0           0          0  ...      1       0   
Review #2: Negative       0           1          0  ...      0       0   
Review #3: Negative       0           0          0  ...      0       0   
Review #4: Negative       0           0          0  ...      0       0   
Review #5: Negative       0  

# Code for later

## Create Term Document Matrix From a Data Frame

In [ ]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data['Comments'])
ColumnNames=vectorizer.get_feature_names()
#print(ColumnNames)
#print(X)

In [ ]:
TDM=pd.DataFrame(X.toarray(),columns=ColumnNames, index = RowNames)
print(TDM)

## Redo, but with Additional Parameters

In [ ]:
stemmer = SnowballStemmer("english")
analyzer = CountVectorizer().build_analyzer()

def stemmed_words(doc):
    return (stemmer.stem(w) for w in analyzer(doc))

In [ ]:
vectorizer = CountVectorizer(stop_words='english', analyzer=stemmed_words, min_df = 2)
X = vectorizer.fit_transform(data['Comments'])
ColumnNames=vectorizer.get_feature_names()
#print(ColumnNames)
#print(X)

In [ ]:
TDM=pd.DataFrame(X.toarray(),columns=ColumnNames, index = RowNames)
print(TDM)

## Redo, but with Additional Parameters

In [ ]:
stemmer = SnowballStemmer("english")
analyzer = CountVectorizer().build_analyzer()

def stemmed_words(doc):
    return (stemmer.stem(w) for w in analyzer(doc))

In [ ]:
vectorizer = CountVectorizer(stop_words='english', analyzer=stemmed_words, min_df = 2)
X = vectorizer.fit_transform(data['Comments'])
ColumnNames=vectorizer.get_feature_names()
#print(ColumnNames)
#print(X)

In [ ]:
TDM=pd.DataFrame(X.toarray(),columns=ColumnNames, index = RowNames)
print(TDM)

## Create Term Document Matrix From a Corpus

In [ ]:
vectorizer = CountVectorizer(input='filename')
X = vectorizer.fit_transform(FilePaths)
ColumnNames=vectorizer.get_feature_names()
#print(ColumnNames)
#print(X)

In [ ]:
TDM=pd.DataFrame(X.toarray(),columns=ColumnNames, index = FileNames)
print(TDM)

## Create a Corpus from a Data Frame

In [ ]:
##Converts each row in the DataFrame's Column 'Comments' into its own individual file to create a corpus

i = 1
for row in data.index:
    if data.iloc[row-1,1] == 'Positive':
        file_title = '/Users/gml/Desktop/test2/PositiveReview{}.txt'.format(i)
        np.asarray(data.values[row-1][0],dtype=np.object).tofile(file_title, sep=",", format="%s")
    else:
        file_title = '/Users/gml/Desktop/test2/NegativeReview{}.txt'.format(i)
        np.asarray(data.values[row-1][0],dtype=np.object).tofile(file_title, sep=",", format="%s")
    i+=1

In [ ]:
path3 ='/Users/gml/Desktop/test2'
FullFileName=os.listdir(path3)
print(FullFileName)

## Collect File Names from Corpus

In [ ]:
path2 ='/Users/gml/Desktop/test1'
FullFileName=os.listdir(path2)
print(FullFileName)

In [ ]:
FilePaths=[]
FileNames=[]

for name in os.listdir(path2):
    if name != '.DS_Store': #Mac Medata file
        #print(path2+ "\\" + name)
        next=path2+ "/" + name
    
        nextnameL=name.split(".")   ##.txt out of name
        nextname=nextnameL[0]   ## provides file name
    
        FilePaths.append(next)
        FileNames.append(nextname)